# Sentiment Analysis

Obtain textual data for sentiment analysis from financial news sources and social media. Sentiment Analysis: Utilize GPT api to run sentiment analysis on the textual data to extract sentiment scores for each REIT in the office sector.

by Cameron Keith

August 20, 2023

AI ML U Chicago Class

In [12]:
# import and setup necessary libraries
import pandas as pd

In [172]:
# Load the CSV file
data = pd.read_csv('reit_news_collection/output.csv')

# Company name to ticker symbol mapping
names_to_tickers = {
    "Boston Properties Inc.": 'BXP', #
    "Highwoods Properties Inc.": 'HIW', #
    "Kilroy Realty Corporation": 'KRC', #
    "Washington Real Estate Investment Trust": 'WRE',
    "Mack-Cali Realty Corporation": 'CLI',
    "Vornado Realty Trust": 'VNO',
    "Metropolitan Partnership Ltd.": 'MPG',
    "Brookfield Property Partners L.P.": 'BPO',
    "SL Green Realty Corp.": 'SLG', #
    "Brandywine Realty Trust": 'BDN', #
    "Douglas Emmett Inc.": 'DEI',
    "Corporate Office Properties Trust": 'OFC', #
    "Piedmont Office Realty Trust Inc.": 'PDM', #
    "Empire State Realty Trust Inc.": 'ESRT',
    "Equity Commonwealth": 'EQC', #
    "Paramount Group Inc.": 'PGRE', #
    "JBG Smith Properties": 'JBGS',
    "Hudson Pacific Properties Inc.": 'HPP', #
    "American Assets Trust Inc.": 'AAT',
    "Vornado Realty Trust": 'VRE' #
}

# Extract official names and ticker names
official_names= names_to_tickers.keys()
ticker_names = names_to_tickers.values()
print(ticker_names)

# Create a list to store individual company mentions
company_mentions = []

# Iterate through each row in the original DataFrame
for _, row in data.iterrows():
    # Extract companies from the 'companies_mentioned' column
    companies = []
    c_temp = str(row['companies_mentioned']).split(',')

    c1 = []
    counter = 0

    # Split and process company names
    while counter < len(c_temp):
        if counter + 1 < len(c_temp) and c_temp[counter + 1] == ' Inc.':
            c1.append(c_temp[counter] + c_temp[counter + 1])
            counter += 1
        else:
            c1.append(c_temp[counter])
            
        counter += 1
    
    companies = []
    i = 0
    while i < len(c1):
        companies.append(c1[i])
        i += 2

    # Calculate the sentiment score for each company and add to the list
    for company in companies:
        if company in official_names:
            company_mentions.append({
                'month': row['month'],
                'year': row['year'],
                'sentiment_score': row['sentiment_score'],
                'company': names_to_tickers[company]
            })
            # add company to office sector
            company_mentions.append({
                'month': row['month'],
                'year': row['year'],
                'sentiment_score': row['sentiment_score'],
                'company': "Office"
            })

        elif  company in ticker_names:
            company_mentions.append({
                'month': row['month'],
                'year': row['year'],
                'sentiment_score': row['sentiment_score'],
                'company': company
            })
            # add company to office sector
            company_mentions.append({
                'month': row['month'],
                'year': row['year'],
                'sentiment_score': row['sentiment_score'],
                'company': "Office"
            })
    # add entire office sector
    if 'Office' in str(row['sector']):
        company_mentions.append({
            'month': row['month'],
            'year': row['year'],
            'sentiment_score': row['sentiment_score'],
            'company': "Office"
        })

# Create a new DataFrame from the list of dictionaries
company_mentions_df = pd.DataFrame(company_mentions)

# Convert 'month' and 'year' columns to integers
company_mentions_df['month'] = company_mentions_df['month'].astype(int)
company_mentions_df['year'] = company_mentions_df['year'].astype(int)

# Calculate the average sentiment scores per month, year, company
average_scores = company_mentions_df.groupby(['month', 'year', 'company']).mean().round(4)

# Reset the index for better readability
average_scores.reset_index(inplace=True)

# Print unique company names
print(average_scores['company'].unique())

# Load politician insider data
politician_insider_data = pd.read_csv('reit_news_collection/output_politician_data.csv')

# Assign 'companies_mentioned' to 'company' column
politician_insider_data['company'] = politician_insider_data['companies_mentioned']

# Drop unnecessary columns
politician_insider_data.drop(['sector', 'companies_mentioned'], axis=1, inplace=True)

# Calculate and sort average politician sentiment scores
average_politician_scores = politician_insider_data.groupby(['month', 'year', 'company'])['sentiment_score'].mean().reset_index()
pol_out = average_politician_scores.sort_values(by=['company', 'year', 'month'])

# Save politician sentiment scores to CSV
pol_out.to_csv('average_politician_scores.csv', index=False)

# Concatenate company mentions and politician data
articles_plus_politician_data = pd.concat([company_mentions_df, politician_insider_data], ignore_index=True)

# Group and calculate average sentiment scores
articles_plus_politician_data_grouped = articles_plus_politician_data.groupby(['month', 'year', 'company'])['sentiment_score'].mean().round(4).reset_index()
out = articles_plus_politician_data_grouped.sort_values(by=['company', 'year', 'month'])

# Print description of the data
print(out.describe())

# Save office sentiment scores to CSV
out.to_csv('average_office_sentiment_scores.csv', index=False)


dict_values(['BXP', 'HIW', 'KRC', 'WRE', 'CLI', 'VRE', 'MPG', 'BPO', 'SLG', 'BDN', 'DEI', 'OFC', 'PDM', 'ESRT', 'EQC', 'PGRE', 'JBGS', 'HPP', 'AAT'])
['Office' 'EQC' 'OFC' 'PDM' 'BXP' 'KRC' 'VRE' 'HPP' 'BDN' 'HIW' 'PGRE'
 'SLG']
            month        year  sentiment_score
count  266.000000  266.000000       266.000000
mean     5.699248   20.263158         0.544723
std      3.382496    2.603031         0.725113
min      1.000000   13.000000        -1.000000
25%      3.000000   19.000000         0.494475
50%      5.000000   21.000000         0.942700
75%      8.000000   22.000000         0.993975
max     12.000000   23.000000         1.000000
